In [ ]:
# Cell 1 — SparkSession & JDBC Setup
from pyspark.sql import SparkSession
import os

DB_CONFIG = {
    "host":     "localhost",
    "port":     "5432",
    "user":     "postgres",
    "password": "1234",
    "db":       "tourism",
    "driver":   "org.postgresql.Driver"
}

spark = (
    SparkSession.builder
    .appName("04_join")
    .config("spark.jars.packages","org.postgresql:postgresql:42.6.0")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

# Cell 2 — Read yearly tables & join
jdbc_url = f"jdbc:postgresql://{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['db']}"
props = {"user": DB_CONFIG["user"], "password": DB_CONFIG["password"], "driver": DB_CONFIG["driver"]}

occ = spark.read.jdbc(jdbc_url, "tourism_occupancy_yearly", properties=props)
cap = spark.read.jdbc(jdbc_url, "tourism_capacity_yearly",  properties=props)

merged_final = occ.join(cap, on=["geo","year"], how="inner")
print("=== Merged schema ===")
merged_final.printSchema()
print("=== Sample rows ===")
merged_final.show(5,truncate=False)

# note: we do NOT write here, next task will persist
